# Single article extraction

In [1]:
# Load csv to pandas dataframe
import pandas as pd
df = pd.read_csv("gobmx_presidencia_articles.csv")
df.head()

,link,date,title
0,https://www.gob.mx//presidencia/es/articulos/v...,2025-11-18,Versión estenográfica. Renovación del Paquete ...
1,https://www.gob.mx//presidencia/es/articulos/v...,2025-11-18,Versión estenográfica. Conferencia de prensa d...
2,https://www.gob.mx//presidencia/es/articulos/v...,2025-11-17,Versión estenográfica. Conferencia de prensa d...
3,https://www.gob.mx//presidencia/es/articulos/v...,2025-11-16,Versión estenográfica. Chocolate Bienestar. Ce...
4,https://www.gob.mx//presidencia/es/articulos/v...,2025-11-15,Versión estenográfica. Supervisión de obra: Am...


In [4]:
article_link = df.iloc[0]['link']
article_link
# Scrap article content from link 


'https://www.gob.mx//presidencia/es/articulos/version-estenografica-renovacion-del-paquete-contra-la-inflacion-y-la-carestia-pacic-palacio-nacional?idiom=es'

In [9]:
from bs4 import BeautifulSoup
import json
import re

def parse_article(html):
    soup = BeautifulSoup(html, "html.parser")

    # --- Extract Title (h1) ---
    title_tag = soup.find("h1")
    title = title_tag.get_text(strip=True) if title_tag else None

    # --- Extract Subtitle (h2) ---
    subtitle_tag = soup.find("h2")
    subtitle = subtitle_tag.get_text(strip=True) if subtitle_tag else None

    # --- Extract article body ---
    body = soup.find("div", class_="article-body")

    content = []
    if body:
        # Extract only meaningful paragraphs
        paragraphs = body.find_all("p")

        for p in paragraphs:
            text = p.get_text(" ", strip=True)

            # Remove non-content artifacts like &nbsp;, empty strings, etc.
            text = re.sub(r"\s+", " ", text).strip()

            if text and text != "·":  # skip trivial elements
                content.append(text)

    # --- Final JSON structure ---
    result = {
        "title": title,
        "subtitle": subtitle,
        "content": content
    }

    return result


# Example usage
html = """<html> ... your content ... </html>"""   # replace with your page
parsed = parse_article(html)
print(json.dumps(parsed, indent=2, ensure_ascii=False))


{
  "title": null,
  "subtitle": null,
  "content": []
}


In [ ]:
info = parse_article(html)

'MODERADOR:\nEscuchemos el mensaje que nos dirige la Presidenta Constitucional de los Estados Unidos Mexicanos, Doctora Claudia Sheinbaum Pardo.\nPRESIDENTA DE MÉXICO, CLAUDIA SHEINBAUM PARDO:\nMuy buenas tardes a todas, a todos.\nGracias al Gabinete, compañeros y compañeras del Gobierno de México.\nY muchas gracias, de verdad, a todas y a todos los empresarios, representantes de distintas empresas, agroindustriales, pesqueras y de cadenas comerciales.\nEste Paquete Contra la Inflación y la Carestía que se firmó por primera vez en el 2022, cuando venía una escalada inflacionaria producto de la guerra en Ucrania, fue un excelente ejemplo en ese momento de cómo un acuerdo muy importante entre gobierno e iniciativa privada para el beneficio de las familias es factible hacerlo y hay voluntad para ello.\nCuando llegamos al gobierno decidimos mantenerlo, y recibimos de parte de ustedes toda la colaboración, toda la cooperación. Y creo que es un excelente ejemplo de cómo gobierno e iniciativa